In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
pd.read_csv('train.tsv', sep='\t').head()

,docid,rel,quid,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,...,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45
0,1,0,10,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.0
1,2,1,10,0.031310,0.666667,0.50,0.166667,0.033206,0.0,0.0,...,0.646890,0.686107,0.823908,0.750092,0.385426,0.923077,0.086207,0.333333,0.448276,0.0
2,3,1,10,0.078682,0.166667,0.50,0.333333,0.080022,0.0,0.0,...,0.649824,0.578581,0.868557,0.641385,0.010462,0.076923,0.074713,0.833333,0.678161,0.0
3,4,1,10,0.019058,1.000000,1.00,0.500000,0.022591,0.0,0.0,...,0.918403,0.868457,1.000000,0.863460,0.016642,0.153846,0.040230,0.833333,0.896552,0.0
4,5,0,10,0.039477,0.000000,0.75,0.166667,0.040555,0.0,0.0,...,0.565875,0.569440,0.769845,0.646567,0.073711,0.076923,0.034483,0.333333,0.218391,0.0


In [3]:
def read_batches(filename):
    with open(filename, 'r') as handler:
        next(handler)
        x_by_q = defaultdict(lambda: [])
        y_by_q = defaultdict(lambda: [])
        for line in handler:
            splited = list(map(float, line.strip().split()))
            q = splited[2]
            y_by_q[q].append(splited[1])
            x_by_q[q].append(splited[3:])
        X, Y = [], []
        for q in x_by_q:
            X.append(np.array(x_by_q[q]))
            Y.append(np.array(y_by_q[q]))
        return X, Y

def train_test_split(X, Y, test_size=0.2):
    assert len(X) == len(Y)
    permutation = np.random.permutation(len(X))
    split = int(len(permutation) * (1 - test_size))
    train_indices = permutation[:split]
    test_indices = permutation[split:]
    return (
        list(np.array(X)[train_indices]),
        list(np.array(Y)[train_indices]),
        list(np.array(X)[test_indices]),
        list(np.array(Y)[test_indices]),
    )

def generate_data(X, Y):
    permutation = np.random.permutation(len(X))
    index = 0
    while True:
        yield X[permutation[index]], Y[permutation[index]]
        index += 1
        if index >= len(X):
            index = 0

In [4]:
""" Reference from https://gist.github.com/bwhite/3726239
"""

import numpy as np

def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

def get_ranked_relevances(model, X, Y):
    all_ranked_relevances = []
    for x, y in zip(X, Y):
        prediction = model.predict(x)
        assert len(prediction) == len(y)
        pairs = list(zip(prediction, y))
        ranked_relevances = [
            y
            for prediction, y in sorted(pairs, key=lambda pair: -pair[0])
        ]
        all_ranked_relevances.append(ranked_relevances)
    return all_ranked_relevances

In [6]:
X, Y = read_batches('train.tsv')
train_X, train_Y, test_X, test_Y = train_test_split(X, Y)
features_num = 46

In [7]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Lambda
from keras.optimizers import Adam
import keras.backend as K


def list_net_loss(y_true, y_pred):
    """
    input:
        ytrue: tensor with shape [bs, 1]
        y_pred: tensor with shape [bs, 1]
    """
    y_true = K.squeeze(y_true, axis=1)  # shape: [bs]
    y_pred = K.squeeze(y_pred, axis=1)  # shape: [bs]
        
    y_true_exp = K.exp(y_true)  # shape: [bs]
    y_pred_exp = K.exp(2 * y_pred)  # shape: [bs]
    
    p_y = y_true_exp / K.sum(y_true_exp)  # shape: [bs]
    p_f = y_pred_exp / K.sum(y_pred_exp)  # shape: [bs]
    
    return -K.sum(p_y * K.log(p_f))

x = Input(shape=(features_num,))

ranking_nn = Sequential()
ranking_nn.add(Dense(64))
ranking_nn.add(Activation('tanh'))
ranking_nn.add(Dense(64))
ranking_nn.add(Activation('tanh'))
ranking_nn.add(Dense(64))
ranking_nn.add(Activation('tanh'))
ranking_nn.add(Dense(1))
ranking_nn.add(Activation('tanh'))

rank = ranking_nn(x)

model = Model(inputs=x, outputs=rank)
model.compile(loss=list_net_loss,
              optimizer=Adam(lr=5e-6))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
ranked_relevances = get_ranked_relevances(model, test_X, test_Y)
ndcg = np.array([ndcg_at_k(r, k=5) for r in ranked_relevances]).mean()
print(ndcg)

0.21239309095448444


In [9]:
model.fit_generator(
    generate_data(train_X, train_Y),
    epochs=100,
    steps_per_epoch=len(train_X),
    validation_data=generate_data(test_X, test_Y),
    validation_steps=len(test_X),
    verbose=1
)

Epoch 1/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.4644 - val_loss: 3.7000
Epoch 2/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3855 - val_loss: 3.6776
Epoch 3/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3748 - val_loss: 3.6714
Epoch 4/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3702 - val_loss: 3.6680
Epoch 5/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3674 - val_loss: 3.6659
Epoch 6/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3655 - val_loss: 3.6644
Epoch 7/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3642 - val_loss: 3.6633
Epoch 8/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3632 - val_loss: 3.6625
Epoch 9/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3625 - val_loss: 3.6619
Epoch 10/100
1190/1190 [==============================] - 1s 1ms/step - l

Epoch 80/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3564 - val_loss: 3.6568
Epoch 81/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3564 - val_loss: 3.6568
Epoch 82/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3563 - val_loss: 3.6568
Epoch 83/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3563 - val_loss: 3.6568
Epoch 84/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3563 - val_loss: 3.6568
Epoch 85/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3563 - val_loss: 3.6568
Epoch 86/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3563 - val_loss: 3.6567
Epoch 87/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3562 - val_loss: 3.6567
Epoch 88/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3562 - val_loss: 3.6567
Epoch 89/100
1190/1190 [==============================] - 1s 1ms

In [10]:
ranked_relevances = get_ranked_relevances(model, test_X, test_Y)
ndcg = np.array([ndcg_at_k(r, k=5) for r in ranked_relevances]).mean()
print(ndcg)
ranked_relevances = get_ranked_relevances(model, test_X, test_Y)
ndcg = np.array([ndcg_at_k(r, k=10) for r in ranked_relevances]).mean()
print(ndcg)

0.4503403843902221
0.4839925868421581


In [11]:
model.fit_generator(
    generate_data(train_X, train_Y),
    epochs=100,
    steps_per_epoch=len(train_X),
    validation_data=generate_data(test_X, test_Y),
    validation_steps=len(test_X),
    verbose=1
)

Epoch 1/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3560 - val_loss: 3.6565
Epoch 2/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3560 - val_loss: 3.6565
Epoch 3/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3560 - val_loss: 3.6564
Epoch 4/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3560 - val_loss: 3.6564
Epoch 5/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3560 - val_loss: 3.6564
Epoch 6/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3559 - val_loss: 3.6564
Epoch 7/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3559 - val_loss: 3.6564
Epoch 8/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3559 - val_loss: 3.6564
Epoch 9/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3559 - val_loss: 3.6564
Epoch 10/100
1190/1190 [==============================] - 1s 1ms/step - l

Epoch 80/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 81/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 82/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 83/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 84/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 85/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 86/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 87/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3550 - val_loss: 3.6560
Epoch 88/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3549 - val_loss: 3.6560
Epoch 89/100
1190/1190 [==============================] - 1s 1ms

In [12]:
ranked_relevances = get_ranked_relevances(model, test_X, test_Y)
ndcg = np.array([ndcg_at_k(r, k=5) for r in ranked_relevances]).mean()
print(ndcg)
ranked_relevances = get_ranked_relevances(model, test_X, test_Y)
ndcg = np.array([ndcg_at_k(r, k=10) for r in ranked_relevances]).mean()
print(ndcg)

0.45060973896374334
0.4803527221402283


In [13]:
model.fit_generator(
    generate_data(train_X, train_Y),
    epochs=100,
    steps_per_epoch=len(train_X),
    validation_data=generate_data(test_X, test_Y),
    validation_steps=len(test_X),
    verbose=1
)

Epoch 1/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 2/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 3/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 4/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 5/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 6/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 7/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 8/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 9/100
1190/1190 [==============================] - 1s 1ms/step - loss: 3.3548 - val_loss: 3.6559
Epoch 10/100
1190/1190 [==============================] - 1s 1ms/step - l

Epoch 80/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3542 - val_loss: 3.6558
Epoch 81/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3542 - val_loss: 3.6558
Epoch 82/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3542 - val_loss: 3.6558
Epoch 83/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3542 - val_loss: 3.6558
Epoch 84/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3542 - val_loss: 3.6558
Epoch 85/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3542 - val_loss: 3.6558
Epoch 86/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3542 - val_loss: 3.6558
Epoch 87/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3541 - val_loss: 3.6558
Epoch 88/100
1190/1190 [==============================] - 2s 1ms/step - loss: 3.3541 - val_loss: 3.6558
Epoch 89/100
1190/1190 [==============================] - 2s 1ms

In [14]:
ranked_relevances = get_ranked_relevances(model, test_X, test_Y)
ndcg = np.array([ndcg_at_k(r, k=5) for r in ranked_relevances]).mean()
print(ndcg)
ranked_relevances = get_ranked_relevances(model, test_X, test_Y)
ndcg = np.array([ndcg_at_k(r, k=10) for r in ranked_relevances]).mean()
print(ndcg)

0.45781311836107896
0.48482830677696565


In [30]:
test = pd.read_csv('test.tsv', sep='\t')
test.head()
res = test[['docid', 'quid', 'feature_0']].copy()
res['feature_0'] = model.predict(test[test.columns.delete((0, 1))])
res.head()
csv = res.sort_values(['quid', 'feature_0'], ascending=False)[['docid', 'quid']]
csv.to_csv('rank_net_tanh.csv', index=False)